##  “Are there synergies and tradeoffs in sustainable heating from cleaner stoves and home insulation? Evidence from air pollution control policies in southern Chile” (Ref.: ENEECO-D-24-01813)



###  Table 4

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
# Set datetime string
from datetime import datetime
from sklearn.metrics import pairwise_distances
datetime_string = datetime.now().strftime("%Y%m%d_%H%M%S")

#### Function to extract results

In [2]:
# Function to extract T1ON results
def extract_t1on_results(results, model_name):
    """
    Extract T1ON parameter, standard error, t-stat, p-value, and confidence intervals from PanelOLS results.
    """
    param = results.params["T1ON"]
    std_err = results.std_errors["T1ON"]
    t_stat = results.tstats["T1ON"]
    p_value = results.pvalues["T1ON"]
    ci_lower = param - 1.96 * std_err
    ci_upper = param + 1.96 * std_err

    return {
        "Model": model_name,
        "PELLET*ON Coefficient": param,
        "Std. Error": std_err,
        "T-stat": t_stat,
        "P-value": p_value,
        "CI Lower": ci_lower,
        "CI Upper": ci_upper,
    }

# Initialize a DataFrame to store results
t1on_results_df = pd.DataFrame()


 #### Table 4. column 1.  The Effect of Pellet Stoves on PM2.5 Concentration (hourly observations)  - without matching

In [3]:
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

from linearmodels.panel import PanelOLS
# Define independent variables and dependent variable
dependent_var = "log_PMi"
independent_vars = ["ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"]
# Fixed Effects Model
model = PanelOLS.from_formula(
    #f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects + TimeEffects",
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 1 - Without Matching")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4847
Estimator:                   PanelOLS   R-squared (Between):              0.7651
No. Observations:               17687   R-squared (Within):               0.4847
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7625
Time:                        02:04:23   Log-likelihood                   -2770.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2712.9
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs:                       49.000   F-statistic (robust):             173.08
                            

 #### Table 4. column 1.  The Effect of Pellet Stoves on PM2.5 Concentration (hourly observations)  - with matching

#### Perform nearest neighbor matching (neighbors = 2) 

In [4]:
# Perform nearest neighbor matching
neighbors = 2
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)

# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 2, Nearest Neighbors Matching, n = 2")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5
                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.5098
Estimator:                   PanelOLS   R-squared (Between):              0.7820
No. Observations:               17687   R-squared (Within):               0.5098
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7792
Time:                        02:04:23   Log-likelihood                   -2129.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2999.4
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs

In [5]:
# Perform nearest neighbor matching
neighbors = 3
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)

# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 3, Nearest Neighbors Matching, n = 3")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5
                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4919
Estimator:                   PanelOLS   R-squared (Between):              0.7653
No. Observations:               17687   R-squared (Within):               0.4919
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7624
Time:                        02:04:23   Log-likelihood                   -2440.6
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2792.1
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs

In [6]:
# Perform nearest neighbor matching
neighbors = 4
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)

# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 4, Nearest Neighbors Matching, n = 4")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5
                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4804
Estimator:                   PanelOLS   R-squared (Between):              0.7438
No. Observations:               17687   R-squared (Within):               0.4804
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7411
Time:                        02:04:23   Log-likelihood                   -2643.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2665.7
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs

In [7]:
# Perform nearest neighbor matching
neighbors = 5
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)

# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 5, Nearest Neighbors Matching, n = 5")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5
                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4836
Estimator:                   PanelOLS   R-squared (Between):              0.7640
No. Observations:               17687   R-squared (Within):               0.4836
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7612
Time:                        02:04:24   Log-likelihood                   -2614.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2700.5
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs

In [8]:
# Perform nearest neighbor matching
neighbors = 6
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)

# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])

model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model 6, Nearest Neighbors Matching, n = 6")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5
                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4860
Estimator:                   PanelOLS   R-squared (Between):              0.7802
No. Observations:               17687   R-squared (Within):               0.4860
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7772
Time:                        02:04:24   Log-likelihood                   -2623.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2726.1
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs

In [9]:
t1on_results_df 

,Model,PELLET*ON Coefficient,Std. Error,T-stat,P-value,CI Lower,CI Upper
0,model 1 - Without Matching,-0.127517,0.022136,-5.760743,8.516641e-09,-0.170903,-0.084132
1,"model 2, Nearest Neighbors Matching, n = 2",-0.092523,0.024358,-3.798535,1.460518e-04,-0.140264,-0.044782
2,"model 3, Nearest Neighbors Matching, n = 3",-0.097194,0.025724,-3.778353,1.583980e-04,-0.147613,-0.046775
3,"model 4, Nearest Neighbors Matching, n = 4",-0.103776,0.025298,-4.102129,4.112390e-05,-0.153360,-0.054192
4,"model 5, Nearest Neighbors Matching, n = 5",-0.102311,0.025744,-3.974199,7.090315e-05,-0.152768,-0.051853
5,"model 6, Nearest Neighbors Matching, n = 6",-0.101040,0.025101,-4.025353,5.713230e-05,-0.150237,-0.051842


Kernel Matching

In [10]:
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
variables_to_keep = ["ID", "T1", "Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
df = df[variables_to_keep]
# Impute missing values with the median
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    df[col] = df[col].fillna(df[col].median())
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = df[covariates]
y = df["T1"]
# Standardize covariates
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]

# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Perform kernel matching using Gaussian kernel
bandwidth = 0.1  # Adjust bandwidth as needed
distances = pairwise_distances(treated[["propensity_score"]], control[["propensity_score"]], metric="euclidean")
kernel_weights = np.exp(-distances**2 / (2 * bandwidth**2))
# Assign weights for control units
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0  # Full weight for treated units
for i, treated_idx in enumerate(treated.index):
    for j, control_idx in enumerate(control.index):
        df.loc[control_idx, "_weight"] += kernel_weights[i, j]
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
# Save weights to a file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)
# Load the original dataset for panel regression
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])
# Fixed Effects Model without weights
dependent_var = "log_PMi"
independent_vars = ["ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"]

# Fixed Effects Model with weights
model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model Kernel Matching, bandwidth = 0.1")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4315
Estimator:                   PanelOLS   R-squared (Between):              0.6731
No. Observations:               17687   R-squared (Within):               0.4315
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.6706
Time:                        02:04:27   Log-likelihood                   -3368.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2188.4
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs:                       49.000   F-statistic (robust):             67.545
                            

In [11]:
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
variables_to_keep = ["ID", "T1", "Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
df = df[variables_to_keep]
# Impute missing values with the median
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    df[col] = df[col].fillna(df[col].median())
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = df[covariates]
y = df["T1"]
# Standardize covariates
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]

# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Perform kernel matching using Gaussian kernel
bandwidth = 0.2  # Adjust bandwidth as needed
distances = pairwise_distances(treated[["propensity_score"]], control[["propensity_score"]], metric="euclidean")
kernel_weights = np.exp(-distances**2 / (2 * bandwidth**2))
# Assign weights for control units
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0  # Full weight for treated units
for i, treated_idx in enumerate(treated.index):
    for j, control_idx in enumerate(control.index):
        df.loc[control_idx, "_weight"] += kernel_weights[i, j]
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
# Save weights to a file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)
# Load the original dataset for panel regression
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])
# Fixed Effects Model without weights
dependent_var = "log_PMi"
independent_vars = ["ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"]

# Fixed Effects Model with weights
model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model Kernel Matching, bandwidth = 0.2")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4413
Estimator:                   PanelOLS   R-squared (Between):              0.7011
No. Observations:               17687   R-squared (Within):               0.4413
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.6985
Time:                        02:04:32   Log-likelihood                   -3274.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2278.0
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs:                       49.000   F-statistic (robust):             79.159
                            

In [12]:
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
variables_to_keep = ["ID", "T1", "Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
df = df[variables_to_keep]
# Impute missing values with the median
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    df[col] = df[col].fillna(df[col].median())
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = df[covariates]
y = df["T1"]
# Standardize covariates
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]

# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Perform kernel matching using Gaussian kernel
bandwidth = 0.3  # Adjust bandwidth as needed
distances = pairwise_distances(treated[["propensity_score"]], control[["propensity_score"]], metric="euclidean")
kernel_weights = np.exp(-distances**2 / (2 * bandwidth**2))
# Assign weights for control units
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0  # Full weight for treated units
for i, treated_idx in enumerate(treated.index):
    for j, control_idx in enumerate(control.index):
        df.loc[control_idx, "_weight"] += kernel_weights[i, j]
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
# Save weights to a file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)
# Load the original dataset for panel regression
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
relevant_vars = ["ID","IDHour","T1ON", "log_PMo", "log_To", "DayExp", "Cycle","ON", "log_PMi"]
# Filter the DataFrame to keep only these columns
df = df[relevant_vars]
df= df.dropna(subset=["log_PMi","ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"])
# Merge weights into original data
weights_df = pd.read_csv(weights_path)
df = pd.merge(df, weights_df, on="ID", how="left")
# Save merged dataset
panel_data_path = f"PanelBDTemuco_{datetime_string}.csv"
df.to_csv(panel_data_path, index=False)
# Prepare for panel regression
df = df.sort_values(by=["ID", "IDHour"])
df = df.set_index(["ID", "IDHour"])
# Fixed Effects Model without weights
dependent_var = "log_PMi"
independent_vars = ["ON", "T1ON", "log_PMo", "log_To", "DayExp", "Cycle"]

# Fixed Effects Model with weights
model = PanelOLS.from_formula(
    f"{dependent_var} ~ { ' + '.join(independent_vars)} + EntityEffects",
    data=df,
    weights=df["weight"],
    drop_absorbed=True
)
results = model.fit(cov_type="clustered", cluster_entity=True)
# Extract T1ON results for the current model
t1on_result = extract_t1on_results(results, model_name="model Kernel Matching, bandwidth = 0.3")
t1on_results_df = pd.concat([t1on_results_df, pd.DataFrame([t1on_result])], ignore_index=True)
print(results.summary)



                          PanelOLS Estimation Summary                           
Dep. Variable:                log_PMi   R-squared:                        0.4470
Estimator:                   PanelOLS   R-squared (Between):              0.7143
No. Observations:               17687   R-squared (Within):               0.4470
Date:                Sun, Dec 08 2024   R-squared (Overall):              0.7116
Time:                        02:04:37   Log-likelihood                   -3217.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2331.2
Entities:                         379   P-value                           0.0000
Avg Obs:                       46.668   Distribution:                 F(6,17302)
Min Obs:                       7.0000                                           
Max Obs:                       49.000   F-statistic (robust):             85.026
                            

In [13]:
t1on_results_df = t1on_results_df.drop(index=[1,2, 3, 4, 5])

In [14]:
t1on_results_df

,Model,PELLET*ON Coefficient,Std. Error,T-stat,P-value,CI Lower,CI Upper
0,model 1 - Without Matching,-0.127517,0.022136,-5.760743,8.516641e-09,-0.170903,-0.084132
6,"model Kernel Matching, bandwidth = 0.1",-0.124490,0.024337,-5.115189,3.167751e-07,-0.172191,-0.076789
7,"model Kernel Matching, bandwidth = 0.2",-0.126606,0.023205,-5.455863,4.940471e-08,-0.172088,-0.081123
8,"model Kernel Matching, bandwidth = 0.3",-0.128121,0.022759,-5.629414,1.836286e-08,-0.172730,-0.083513


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
# Set datetime string
from datetime import datetime
from sklearn.metrics import pairwise_distances
datetime_string = datetime.now().strftime("%Y%m%d_%H%M%S")

# Perform nearest neighbor matching
neighbors = 4
# Load the dataset
file_path = "BDTemuco.dta"  # Update with the correct path
df = pd.read_stata(file_path)
# Filter data
df = df[df['tper'] < 2]
# Define the variables to keep
variables_to_keep = ["ID","T1","Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
# Filter the DataFrame to keep only these columns
df = df[variables_to_keep]

# Impute missing values with the median of each column
columns_to_impute = ["Education", "GenderHHead1women", "Age"]
for col in columns_to_impute:
    median_value = df[col].median()  # Calculate the median
    df[col] = df[col].fillna(median_value)  # Replace missing values with the median
# Logistic regression to estimate propensity scores
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]
X = sm.add_constant(df[covariates])
y = df["T1"]
logit_model = sm.Logit(y, X).fit()
df["logoddsT1"] = logit_model.predict(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
logit_model = LogisticRegression(max_iter=500)
df["propensity_score"] = logit_model.fit(X_scaled, y).predict_proba(X_scaled)[:, 1]
# Separate treated and control groups
treated = df[df["T1"] == 1]
control = df[df["T1"] == 0]

# Apply caliper (0.01)
caliper = 0.01
matches4 = []
nn = NearestNeighbors(n_neighbors=neighbors, metric="euclidean")
nn.fit(control[["propensity_score"]])
distances, indices = nn.kneighbors(treated[["propensity_score"]])
matches4 = []
for i, dists in enumerate(distances):
    matched_indices = indices[i][dists <= caliper]
    if len(matched_indices) > 0:
        for control_index in matched_indices:
            matches4.append((treated.index[i], control.iloc[control_index].name))
# Generate matched DataFrame
matched_pairs = pd.DataFrame(matches4, columns=["treated_index", "control_index"])
# Assign weights
df["_weight"] = 0.0
df.loc[treated.index, "_weight"] = 1.0
for treated_idx in matched_pairs["treated_index"].unique():
    controls = matched_pairs[matched_pairs["treated_index"] == treated_idx]["control_index"]
    df.loc[controls, "_weight"] += 1 / len(controls)
df["weight"] = df["_weight"]
# Replace zero weights with a small positive value
df["weight"] = df["weight"].apply(lambda x: x if x > 0 else 1e-08)
#df["weight"] = df["weight"] / df["weight"].mean()
# Save weights to file
weights_path = f"Weights_{datetime_string}.csv"
df[["ID", "weight"]].to_csv(weights_path, index=False)


Optimization terminated successfully.
         Current function value: 0.660393
         Iterations 5


In [16]:
# Function to calculate Standardized Mean Difference (SMD)
def calculate_smd(treated, control, variable):
    mean_treated = treated[variable].mean()
    mean_control = control[variable].mean()
    std_pooled = np.sqrt(
        (treated[variable].var() + control[variable].var()) / 2
    )
    return (mean_treated - mean_control) / std_pooled

# Define covariates for balance analysis
covariates = ["Education", "GenderHHead1women", "Age", "Disability", "NSize", "Anypersonolder60", "Chamber"]

# Separate treated and control groups before matching
treated_before = df[df["T1"] == 1]
control_before = df[df["T1"] == 0]

# Subset matched pairs for after matching
matched_control_indices = matched_pairs["control_index"].values
control_after = control_before.loc[matched_control_indices]
treated_after = treated_before

# Calculate SMD before and after matching
smd_results = []
for cov in covariates:
    smd_before = calculate_smd(treated_before, control_before, cov)
    smd_after = calculate_smd(treated_after, control_after, cov)
    smd_results.append({"Covariate": cov, "SMD Before": smd_before, "SMD After": smd_after})

# Convert to DataFrame for tabular output
smd_df = pd.DataFrame(smd_results)



In [17]:
# Display the balance table
print(smd_df)

           Covariate  SMD Before  SMD After
0          Education    0.161934   0.097025
1  GenderHHead1women   -0.079849  -0.033056
2                Age    0.291631   0.116805
3         Disability    0.041452  -0.003107
4              NSize   -0.271687   0.000036
5   Anypersonolder60    0.304497   0.106261
6            Chamber    0.212116   0.001474
